# Rizzbot notebook 1: Data Preprocessing pipeline. The purpose of this notebook is to set up the pinecone index, scrape the YT videos from the channel of choice, convert 
# the YT videos into Vectors and then store the vectors into the pinecone index for later use. For each step in between, we save the files temporarily on my local device and then transfer them to a bucket storage in AWS. 

In [2]:
# Set up and connect to the environment for Pinecone
import os
from dotenv import load_dotenv, find_dotenv
from pinecone import Pinecone, ServerlessSpec

# Load environment variables from .env file
_ = load_dotenv(find_dotenv())
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Initialize Pinecone client
pc = Pinecone(api_key = PINECONE_API_KEY)

# Create a new index
index_name = "rizzbot"
dimension = 1024 # Dimension of the embedding model text-embedding-3-large output dimension

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Connect to the index
index = pc.Index(index_name)

print(f"Connected to index: {index_name}")

Connected to index: rizzbot


In [ ]:
# Script with batch processing and timeout, downloaded 200 videos after which I manually interrupted the script. Because 200 videos is enough and youtube has started blocking the script
import subprocess
import boto3
import time
import json
import sys
from pathlib import Path


# === CONFIG ===
CHANNEL_URL = "https://www.youtube.com/user/Charismaoncommand/videos"
TEMP_DIR = (Path.cwd() / "tmp_files" / "rizzbot_downloads").resolve()
S3_BUCKET = "rizzbot-temp-storage"
S3_PREFIX = "rizzbot/Audio"
YT_TIMEOUT = 600  # 10 minutes max per video
SLEEP_MIN = 2
SLEEP_MAX = 5
MAX_VIDEOS = 397  # actual video count

session = boto3.Session(profile_name="rizzbot")
s3_client = session.client("s3")
TEMP_DIR.mkdir(parents=True, exist_ok=True)

def log(msg):
    print(f"[RIZZBOT] {msg}")

def get_video_list(channel_url, max_videos):
    """
    Use yt-dlp to fetch video URLs from the channel
    """
    try:
        result = subprocess.run(
            ["yt-dlp", "--flat-playlist", "--dump-single-json", "--playlist-end", str(max_videos), channel_url],
            capture_output=True, text=True, check=True
        )
        playlist = json.loads(result.stdout)  # yt-dlp returns JSON, safer to parse but yt-dlp can be inconsistent
        entries = playlist.get("entries", [])
        video_urls = [f"https://www.youtube.com/watch?v={entry['id']}" for entry in entries]
        return video_urls
    except subprocess.CalledProcessError as e:
        log(f"Failed to fetch video list: {e.stderr}")
        sys.exit(1)

def download_and_upload_video(video_url, index, total):
    """
    Download single video audio and upload to S3
    """
    log(f"Processing video {index + 1}/{total}: {video_url}")

    output_path = str(TEMP_DIR / "%(title)s.%(ext)s")

    ytdlp_cmd = [
        "yt-dlp",
        "--extract-audio",
        "--audio-format", "mp3",
        "--ignore-errors",
        "--socket-timeout", "10",
        "--sleep-interval", str(SLEEP_MIN),
        "--max-sleep-interval", str(SLEEP_MAX),
        "--ffmpeg-location", r"C:\Users\karel\ffmpeg-7.1.1-full_build\bin",
        "-o", output_path,
        video_url
    ]

    try:
        subprocess.run(ytdlp_cmd, check=True, timeout=YT_TIMEOUT, capture_output=True, text=True)
    except subprocess.TimeoutExpired:
        log(f"Timeout expired for video: {video_url}. Skipping.")
        return
    except subprocess.CalledProcessError as e:
        log(f"yt-dlp failed for video {video_url}: {e.stderr}")
        return

    # Upload all mp3 files created (usually 1)
    for audio_file in TEMP_DIR.glob("*.mp3"):
        s3_key = f"{S3_PREFIX}/{audio_file.name}"
        try:
            s3_client.upload_file(str(audio_file), S3_BUCKET, s3_key)
            log(f"Uploaded {audio_file.name} to s3://{S3_BUCKET}/{s3_key}")
        except Exception as e:
            log(f"Failed to upload {audio_file.name}: {e}")
        audio_file.unlink()

def process_channel_videos():
    video_urls = get_video_list(CHANNEL_URL, MAX_VIDEOS)
    total = len(video_urls)
    log(f"Found {total} videos to process.")

    for index, video_url in enumerate(video_urls):
        download_and_upload_video(video_url, index, total)

    log("All videos processed.")

# === RUN ===
if __name__ == "__main__":
    process_channel_videos()


[RIZZBOT] Found 397 videos to process.
[RIZZBOT] Processing video 1/397: https://www.youtube.com/watch?v=3r8SMMm_IkQ
[RIZZBOT] Uploaded This Energy Is Missing In Modern Men.mp3 to s3://rizzbot-temp-storage/rizzbot/Audio/This Energy Is Missing In Modern Men.mp3
[RIZZBOT] Processing video 2/397: https://www.youtube.com/watch?v=YAX6vZ9xi8w
[RIZZBOT] Uploaded Old vs new Jordan Peterson… What Went Wrong？.mp3 to s3://rizzbot-temp-storage/rizzbot/Audio/Old vs new Jordan Peterson… What Went Wrong？.mp3
[RIZZBOT] Processing video 3/397: https://www.youtube.com/watch?v=kg7v-f9r6nc
[RIZZBOT] Uploaded 5 Simple Ways To Command More Respect.mp3 to s3://rizzbot-temp-storage/rizzbot/Audio/5 Simple Ways To Command More Respect.mp3
[RIZZBOT] Processing video 4/397: https://www.youtube.com/watch?v=lVxGwDeYmLw
[RIZZBOT] Uploaded Be Like This To Naturally Attract Women.mp3 to s3://rizzbot-temp-storage/rizzbot/Audio/Be Like This To Naturally Attract Women.mp3
[RIZZBOT] Processing video 5/397: https://www.you

KeyboardInterrupt: 

In [ ]:
# This script downloads audio files from S3, transcribes them using OpenAI's Whisper model, and uploads the transcripts back to a different folder in S3.


import os
import openai
import boto3
import json
import time
import tempfile
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from botocore.exceptions import BotoCoreError, ClientError
import traceback

_ = load_dotenv(find_dotenv())

# === CONFIG ===
S3_BUCKET = "rizzbot-temp-storage"
AUDIO_PREFIX = "rizzbot/Audio/"
TRANSCRIPT_PREFIX = "rizzbot/Transcripts"
api_key = os.getenv("OPENAI_TEST_KEY_KdR")
openai.api_key = api_key
MAX_RETRIES = 3
MAX_WORKERS = 3  # Number of concurrent transcriptions

session = boto3.Session(profile_name="rizzbot")
s3_client = session.client("s3")


def log(msg):
    print(f"[RIZZBOT] {msg}")


def download_file_with_retry(bucket, key, local_path):
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            s3_client.download_file(bucket, key, str(local_path))
            return
        except (BotoCoreError, ClientError) as e:
            log(f"Download failed (attempt {attempt}): {e}")
            time.sleep(2 * attempt)
    raise RuntimeError(f"Failed to download {key} after {MAX_RETRIES} attempts.")


def upload_file_with_retry(bucket, key, body_bytes):
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            s3_client.put_object(Bucket=bucket, Key=key, Body=body_bytes)
            return
        except (BotoCoreError, ClientError) as e:
            log(f"Upload failed (attempt {attempt}): {e}")
            time.sleep(2 * attempt)
    raise RuntimeError(f"Failed to upload {key} after {MAX_RETRIES} attempts.")


def transcribe_audio_from_s3(s3_bucket, s3_key):
    # Use a unique tmp file path
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp_file:
        local_file = Path(tmp_file.name)

    download_file_with_retry(s3_bucket, s3_key, local_file)

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            with open(local_file, "rb") as audio_file:
                transcript_resp = openai.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file,
                    response_format="json"
                )
            break
        except openai.OpenAIError as e:
            log(f"Transcription failed (attempt {attempt}): {e}")
            time.sleep(2 * attempt)
    else:
        local_file.unlink(missing_ok=True)
        raise RuntimeError(f"Failed to transcribe {s3_key} after {MAX_RETRIES} attempts.")

    local_file.unlink(missing_ok=True)
    return transcript_resp


def save_transcript_to_s3(transcript_data, s3_bucket, s3_key):
    json_bytes = json.dumps(transcript_data.model_dump()).encode('utf-8')
    upload_file_with_retry(s3_bucket, s3_key, json_bytes)
    log(f"Transcript raw: {transcript_data.model_dump()}")


def process_single_audio(s3_key):
    log(f"Starting transcription for {s3_key}")
    try:
        transcript_data = transcribe_audio_from_s3(S3_BUCKET, s3_key)
        base_name = Path(s3_key).stem
        transcript_s3_key = f"{TRANSCRIPT_PREFIX}/{base_name}.json"
        save_transcript_to_s3(transcript_data, S3_BUCKET, transcript_s3_key)
        log(f"Completed transcription for {s3_key}")
    except Exception as e:
        log(f"Transcription failed for {s3_key}: {e}")
        log(traceback.format_exc())


def process_all_audio_files():
    log(f"Listing audio files in s3://{S3_BUCKET}/{AUDIO_PREFIX}")

    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=S3_BUCKET, Prefix=AUDIO_PREFIX)

    mp3_files = []
    for page in pages:
        mp3_files.extend(
            obj["Key"] for obj in page.get("Contents", []) if obj["Key"].endswith(".mp3")
        )

    if not mp3_files:
        log("No MP3 files found to process.")
        return

    log(f"Found {len(mp3_files)} MP3 files. Starting parallel transcription...")

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(process_single_audio, key): key for key in mp3_files}
        for future in as_completed(futures):
            key = futures[future]
            try:
                future.result()
            except Exception as e:
                log(f"Unhandled failure for {key}: {e}")
                log(traceback.format_exc())


if __name__ == "__main__":
    process_all_audio_files()


[RIZZBOT] Listing audio files in s3://rizzbot-temp-storage/rizzbot/Audio/
[RIZZBOT] Found 179 MP3 files. Starting parallel transcription...
[RIZZBOT] Starting transcription for rizzbot/Audio/$120,000 Was Stolen From Me... It Was My ＂Friend＂.mp3
[RIZZBOT] Starting transcription for rizzbot/Audio/3 Common Jokes That Make People INSTANTLY Like You Less.mp3
[RIZZBOT] Starting transcription for rizzbot/Audio/3 Jokes To Make Rude People Regret Insulting You.mp3
[RIZZBOT] Transcript raw: {'text': "Very few people can control a conversation better than a comedian. Right. What we've got here is a person who's under mistaken belief that they're funny. Comedians have arsenals of jokes that put rude people in their place and redirect the flow of conversation. Even when there isn't a conflict, when a joke lands and everybody laughs, there's that period of space where you can lead the conversation wherever you like. So in this video, we're going to look at three specific types of jokes that Russell 

In [8]:
# This script chunks and embeds transcripts from S3, uploading them to Pinecone.
# It uses OpenAI's embedding model and handles retries with exponential backoff.
# Vector IDs are sequential (emb-0001, emb-0002, etc.) for easy retrieval.

import json
import time
import argparse
import yaml
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

import boto3
import openai
from pinecone import Pinecone
import tiktoken
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

# --- INIT ---
session = boto3.Session(profile_name="rizzbot", region_name="eu-north-1")
s3_client = session.client("s3")

# Global counter for vector IDs with thread safety
vector_counter = 0
counter_lock = Lock()

def get_next_vector_id():
    """Generate sequential vector IDs like emb-0001, emb-0002, etc."""
    global vector_counter
    with counter_lock:
        vector_counter += 1
        return f"emb-{vector_counter:04d}"

# --- RETRY DECORATOR ---
def retry_with_backoff(max_retries=5, initial_delay=1.0, backoff_factor=2.0):
    def decorator(func):
        def wrapper(*args, **kwargs):
            delay = initial_delay
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_retries - 1:
                        raise
                    print(f"Retry {attempt+1}/{max_retries} after error: {e}")
                    time.sleep(delay)
                    delay *= backoff_factor
        return wrapper
    return decorator

# --- CHUNKING ---
def chunk_text_tokens(text, chunk_size, overlap, tokenizer):
    tokens = tokenizer.encode(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
        start += chunk_size - overlap
    return chunks

# --- LOAD TRANSCRIPTS ---
def list_transcript_keys(s3_client, bucket, prefix):
    keys = []
    continuation_token = None
    while True:
        kwargs = {'Bucket': bucket, 'Prefix': prefix}
        if continuation_token:
            kwargs['ContinuationToken'] = continuation_token
        response = s3_client.list_objects_v2(**kwargs)
        keys.extend([item['Key'] for item in response.get('Contents', []) if item['Key'].endswith('.json')])
        if response.get('IsTruncated'):
            continuation_token = response.get('NextContinuationToken')
        else:
            break
    return keys

def load_transcript(s3_client, bucket, key):
    response = s3_client.get_object(Bucket=bucket, Key=key)
    body = response['Body'].read()
    data = json.loads(body)
    return data.get('text', '')

# --- EMBEDDING + UPLOAD ---
@retry_with_backoff()
def embed_chunk(chunk, embed_model, embed_dimensions):
    response = openai.embeddings.create(
        input=chunk,
        model=embed_model, 
        dimensions=embed_dimensions
    )
    embedding = response.data[0].embedding
    return embedding

@retry_with_backoff()
def upsert_batch(index, batch):
    index.upsert(batch)

# --- PROCESS TRANSCRIPT ---
def process_transcript(s3_client, key, config, tokenizer, index):
    text = load_transcript(s3_client, config['s3_bucket'], key)
    chunks = chunk_text_tokens(text, config['chunk_size'], config['overlap'], tokenizer)
    vectors = []
    
    # Extract filename from S3 key for better metadata
    filename = key.split('/')[-1].replace('.json', '')
    
    for chunk_idx, chunk in enumerate(chunks):
        embedding = embed_chunk(chunk, config['embed_model'], config['embed_dimensions'])
        vector_id = get_next_vector_id()
        
        vectors.append({
            'id': vector_id,
            'values': embedding,
            'metadata': {
                's3_key': key,
                'filename': filename,
                'chunk_index': chunk_idx,
                'chunk_count': len(chunks),
                'text_sample': chunk[:200],  # Increased sample size
                'full_text': chunk,  # Store full chunk text for easy access
                'token_count': len(tokenizer.encode(chunk))
            }
        })
    
    # Batch upload in groups
    for i in range(0, len(vectors), config['pinecone_batch_size']):
        batch = vectors[i:i + config['pinecone_batch_size']]
        upsert_batch(index, batch)
    
    return len(vectors)  # Return number of vectors created

# --- HELPER FUNCTIONS FOR RETRIEVAL ---
def get_vector_id_range(start_id, end_id):
    """Generate a list of vector IDs in range for easy retrieval"""
    start_num = int(start_id.split('-')[1])
    end_num = int(end_id.split('-')[1])
    return [f"emb-{i:04d}" for i in range(start_num, end_num + 1)]

def save_vector_mapping(vectors_created, config_path):
    """Save a mapping file to track which vectors belong to which transcripts"""
    mapping_file = Path(config_path).parent / 'vector_mapping.json'
    mapping = {
        'total_vectors': vector_counter,
        'last_vector_id': f"emb-{vector_counter:04d}",
        'vectors_per_transcript': vectors_created
    }
    with open(mapping_file, 'w') as f:
        json.dump(mapping, f, indent=2)
    print(f"Vector mapping saved to {mapping_file}")

# --- MAIN ---
def main(config_path, s3_client):
    global vector_counter
    
    # Load config
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)

    openai.api_key = config['openai_api_key']
    pc = Pinecone(api_key=config['pinecone_api_key'])
    index = pc.Index(config['pinecone_index'])
    tokenizer = tiktoken.encoding_for_model(config['embed_model'])

    keys = list_transcript_keys(s3_client, config['s3_bucket'], config['transcripts_prefix'])
    
    # Track vectors created per transcript
    vectors_created = {}
    total_vectors = 0

    with ThreadPoolExecutor(max_workers=config['max_workers']) as executor:
        futures = {executor.submit(process_transcript, s3_client, key, config, tokenizer, index): key for key in keys}
        for future in tqdm(as_completed(futures), total=len(futures), desc='Processing transcripts'):
            try:
                key = futures[future]
                num_vectors = future.result()
                vectors_created[key] = num_vectors
                total_vectors += num_vectors
                print(f"Processed {key}: {num_vectors} vectors")
            except Exception as e:
                print(f"Failed processing {futures[future]}: {e}")
    
    print(f"\nTotal vectors created: {total_vectors}")
    print(f"Vector ID range: emb-0001 to emb-{vector_counter:04d}")
    
    # Save mapping for future reference
    save_vector_mapping(vectors_created, config_path)

# --- RETRIEVAL HELPER FUNCTIONS ---
def retrieve_all_vectors(config_path):
    """Retrieve all vectors for clustering - call this in your next notebook"""
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    pc = Pinecone(api_key=config['pinecone_api_key'])
    index = pc.Index(config['pinecone_index'])
    
    # Load vector mapping to know the range
    mapping_file = Path(config_path).parent / 'vector_mapping.json'
    with open(mapping_file, 'r') as f:
        mapping = json.load(f)
    
    total_vectors = mapping['total_vectors']
    vector_ids = [f"emb-{i:04d}" for i in range(1, total_vectors + 1)]
    
    # Fetch in batches (Pinecone has limits on batch size)
    batch_size = 100
    all_vectors = []
    
    for i in range(0, len(vector_ids), batch_size):
        batch_ids = vector_ids[i:i + batch_size]
        response = index.fetch(ids=batch_ids)
        all_vectors.extend(list(response['vectors'].values()))
    
    return all_vectors

def retrieve_vectors_by_transcript(config_path, s3_key):
    """Retrieve all vectors for a specific transcript"""
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    pc = Pinecone(api_key=config['pinecone_api_key'])
    index = pc.Index(config['pinecone_index'])
    
    # Query by metadata
    response = index.query(
        filter={"s3_key": s3_key},
        top_k=10000,  # Adjust based on expected chunk count
        include_metadata=True,
        include_values=True
    )
    
    return response['matches']

# --- CLI ---
if __name__ == '__main__':
    # Check if running in Jupyter notebook
    try:
        get_ipython()
        # Running in Jupyter - use default config path
        config_path = str(Path.cwd() / 'config.yaml')
        print(f"Running in Jupyter notebook, using config: {config_path}")
        main(config_path, s3_client)
    except NameError:
        # Running from command line - use argparse
        parser = argparse.ArgumentParser(description="Rizzbot Chunk + Embed Pipeline")
        parser.add_argument('--config', type=str, required=True, help='Path to YAML config file')
        args = parser.parse_args()
        main(args.config, s3_client)

Running in Jupyter notebook, using config: c:\Users\karel\Ironhack-Bootcamp-Assignments\Rizzbot\config.yaml


Processing transcripts:   1%|          | 1/179 [00:08<25:20,  8.54s/it]

Processed rizzbot/Transcripts/3 Jokes To Make Rude People Regret Insulting You.json: 3 vectors


Processing transcripts:   1%|          | 2/179 [00:09<11:12,  3.80s/it]

Processed rizzbot/Transcripts/3 Common Jokes That Make People INSTANTLY Like You Less.json: 4 vectors
Processed rizzbot/Transcripts/3 Psychological Tricks To Instantly Be More Attractive.json: 3 vectors


...

Processed rizzbot/Transcripts/Women are easily attracted to men who have these habits.json: 6 vectors
Processed rizzbot/Transcripts/Words That Win： How To Instantly Influence Anyone (use ethically).json: 5 vectors


Processing transcripts:  99%|█████████▉| 178/179 [00:56<00:00,  3.52it/s]

Processed rizzbot/Transcripts/You Think You’re Charismatic… But This Proves You’re Just a People Pleaser.json: 3 vectors


Processing transcripts: 100%|██████████| 179/179 [00:57<00:00,  3.13it/s]

Processed rizzbot/Transcripts/why you feel awkward in conversation.json: 5 vectors

Total vectors created: 741
Vector ID range: emb-0001 to emb-0741
Vector mapping saved to c:\Users\karel\Ironhack-Bootcamp-Assignments\Rizzbot\vector_mapping.json
